This example shows you how to use the reader to read in, pre-process and reconstruct a single slice. It then does the same on the full dataset. It is not intended to be used as an optimised script, but it should demonstrate how to use CIL to get going.

Import the python modules we need

In [ ]:
from cil.framework import ImageGeometry
from cil.utilities.jupyter import islicer
from cil.utilities.display import show2D, show_geometry
from cil.processors import CentreOfRotationCorrector, TransmissionAbsorptionConverter
from cil.recon import FBP
from cil.io import NEXUSDataWriter
import numpy as np
import os

from scripts.ESRF_ID15_DataReader import ESRF_ID15_DataReader
from scripts.WeightDuplicateAngles import WeightDuplicateAngles

Set the path to the master hdf5 file and create a reader

In [ ]:
filename = '/data/ESRF/test_data/PC811_1000cycles_absct_final_0001.h5'
reader = ESRF_ID15_DataReader(filename)

You can get the geometry of the scan using `get_geomerty()` as default it'll return the combined geometry. You can pass either scan `dataset_id=1` or `dataset_id=2`to get the individual scans

In [ ]:
geometry = reader.get_geometry()

print(geometry)

show_geometry(geometry)

You can further configure the reader by setting a region of interest `reader.set_roi()`. Currently you can set only a single slice vertically. This will be expanded in the future. Here we set a single slice and check the geometry.


In [ ]:
reader.set_roi(vertical=400)
print(reader.get_geometry())

Now we can read in the data using `reader.read()` as we have set  region of interest we will read only this slice. Again you can specify which dataset with `dataset_id` but the default or `None` will return the combined data.

In [ ]:
data = reader.read(dataset_id=None)
show2D(data)

We can use `TransmissionAbsorptionConverter` to take the -log to convert the data to absorption, we overwrite the original data.

In [ ]:
processor = TransmissionAbsorptionConverter()
processor.set_input(data)
processor.get_output(out=data)
show2D(data)

We centre the dataset. This updates the meta data and is used in the reconstruction. The input data is not padded or cropped. This allows sub pixel offsets. Here we can print the geometry and see `Rotation axis position` has an x component of `+83.78`

In [ ]:
processor = CentreOfRotationCorrector.xcorrelation()
processor.set_input(data)
processor.get_output(out=data)

print(data.geometry)

ToDo: Ring removal example

To reconstruct a slice we create an FBP reconstructor with the dataset and call `run()`

In [ ]:
reco = FBP(data).run()
show2D(reco)

We can set the reconstruction window to remove empty space and shift the object to the centre of the window.

In [ ]:
ig_slice = ImageGeometry(voxel_num_x=1500, voxel_num_y=1500, center_x=0, center_y=50)
reco = FBP(data, ig_slice).run()
show2D(reco)

We have some angles with double the data so we see artifacts. We can find the duplicate angles and weight them appropriately for FBP. Using `weight_duplicate_angles` from `weight_duplicate_angles.py`

In [ ]:
processor = WeightDuplicateAngles()
processor.set_input(data)
processor.get_output(out=data)

show2D(data)

And now we reconstruct the corrected data

In [ ]:
ig_slice = ImageGeometry(1500, 1500, center_x=0, center_y=50)
reco = FBP(data, ig_slice).run()
show2D(reco)

# Processing the full dataset

read in the full dataset by resetting the region of interest in the reader. Then we read it in as before with `reader.read()`

In [ ]:
reader.set_roi(None)
data = reader.read()

We can view the data interactively using `islicer`, by default we scroll through `angles`

In [ ]:
islicer(data, origin='upper-left', size=10)

We call the processors on the dataset using their short form:

In [ ]:
TransmissionAbsorptionConverter()(data, out=data)
CentreOfRotationCorrector.xcorrelation()(data, out = data)
WeightDuplicateAngles()(data, out = data)

And set up out FBP reconstructor with a custom window:

In [ ]:
ig = ImageGeometry(voxel_num_x = 1500, voxel_num_y = 1500, voxel_num_z = 500)
fbp = FBP(data, ig)

Run the reconstructor with `FBP.run()`

In [ ]:
reco = fbp.run()

View the volume using islicer. You can change the slice direction by setting  `direction='vertical'` or `'horizontal_y'` or `'horizontal_x'`

In [ ]:
islicer(reco, direction='horizontal_y', origin='upper-left', size=10)

Save the array as a binary:

In [ ]:
path_out = os.path.abspath('/data/ESRF/test_data/vol_{0}_{1}_{2}'.format(*reco.shape))
#reco.as_array().astype(np.float32).tofile(path_out + '.raw')

Or as a CIL formatted volume - Nexus is a standard derived from hdf5 so you should be able to access it with your HDF5 file browser:

In [ ]:
path_out = os.path.abspath('/data/ESRF/test_data/vol.nxs')
writer = NEXUSDataWriter(data=reco, file_name=path_out, compression=0)
#writer.write()
#%%